### During a discussion with Sinan from Etihad, a significant Fleet Assignment Problem was outlined. In the aviation industry, aircraft require regular maintenance, which can disrupt flight schedules. To ensure operational continuity, alternate aircraft are assigned to cover these routes. The challenge lies in optimizing this assignment, as it is constrained by factors such as pilot availability, aircraft availability and capacity, and cost. This process is currently handled manually. Based on our initial conversation, I propose the following code to address this problem.

In [19]:
from pulp import *

In [24]:
# Define the problem data
flights = ['JFK_LAX', 'JFK_LHR', 'LAX_HND']
fleet_types = ['737', '787']
pilots = ['a', 'b', 'c', 'd']

pilot_qualifications = {
    'a': ['737'],
    'b': ['737', '787'],
    'c': ['787'],
    'd': ['737']
}

fleet_cost = {'737': 50000, '787': 100000}
fleet_capacity = {'737': 150, '787': 300}
flight_demand = {'JFK_LAX': 140, 'JFK_LHR': 145, 'LAX_HND': 250}

# Number of available aircraft of each fleet type
available_fleets = {
    '737': 3,  # We have 3 available 737 aircraft
    '787': 1   # We have 1 available 787 aircraft
}

prob = LpProblem("FleetAndPilotAssignment", LpMinimize)

In [25]:
# Decision Variables
x = LpVariable.dicts("FleetAssignment", [(f, t) for f in flights for t in fleet_types], 0, 1, LpBinary)
y = LpVariable.dicts("PilotAssignment", [(p, f) for p in pilots for f in flights], 0, 1, LpBinary)

# Objective Function: Minimize total cost
prob += lpSum(x[f, t] * fleet_cost[t] for f in flights for t in fleet_types)


In [26]:
# Constraints
# 1. Each flight must be assigned to exactly one fleet type
for f in flights:
    prob += lpSum(x[f, t] for t in fleet_types) == 1

# 2. The assigned fleet must have enough capacity
for f in flights:
    prob += lpSum(x[f, t] * fleet_capacity[t] for t in fleet_types) >= flight_demand[f]

# 3. Each flight must be assigned to exactly one pilot
for f in flights:
    prob += lpSum(y[p, f] for p in pilots) == 1

# 4. A pilot can only be assigned to a flight if their assigned fleet matches their qualification.
# This constraint enforces the link between fleet and pilot assignment.
for f in flights:
    for t in fleet_types:
        for p in pilots:
            if t not in pilot_qualifications[p]:
                prob += x[f, t] + y[p, f] <= 1
                
# 5. A pilot can only be assigned to at most one flight
for p in pilots:
    prob += lpSum(y[p, f] for f in flights) <= 1
    
# 6. The number of flights assigned to a fleet type cannot exceed the number of available aircraft
for t in fleet_types:
    prob += lpSum(x[f, t] for f in flights) <= available_fleets[t]

In [29]:
# Solve the problem
prob.solve()

# Print the results
print("Status:", LpStatus[prob.status])
print(f"\nTotal Minimum Cost: ${prob.objective.value():,.2f}")
print("\nOptimal Assignments:")
for f in flights:
    for t in fleet_types:
        if x[f, t].varValue == 1:
            print(f"- Flight {f} is assigned to a {t}")
    for p in pilots:
        if y[p, f].varValue == 1:
            print(f"  - Pilot {p} is assigned to Flight {f}")


Status: Optimal

Total Minimum Cost: $200,000.00

Optimal Assignments:
- Flight JFK_LAX is assigned to a 737
  - Pilot b is assigned to Flight JFK_LAX
- Flight JFK_LHR is assigned to a 737
  - Pilot d is assigned to Flight JFK_LHR
- Flight LAX_HND is assigned to a 787
  - Pilot c is assigned to Flight LAX_HND
